In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Flatten
from tensorflow.keras.optimizers import SGD

import urllib.parse
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
## Loading data

In [ ]:
def loadFile(name):
    filepath = os.path.join(str(os.getcwd()), name)
    with open(filepath,'r') as f:
        data = f.readlines()
    return [str(urllib.parse.unquote(e)) for e in list(set(data))]

In [ ]:
badQueries = loadFile('dataset/badqueries.txt')

badCount = len(badQueries)

In [ ]:
validQueries = loadFile('dataset/goodqueries.txt')

validCount = len(validQueries)

In [ ]:
queries = badQueries + validQueries

In [ ]:
print("bad: ", badCount)
print("good: ", validCount)
print("all: ", badCount + validCount)

In [ ]:
yBad = [[0, 1] for i in range(0, len(badQueries))]  #labels, 1 for malicious and 0 for clean
yGood = [[1, 0] for i in range(0, len(validQueries))]

In [ ]:
y = yBad + yGood

In [ ]:
## Preparing the dataset

In [ ]:
tokenizer = Tokenizer(filters="", lower=False, char_level=True)

In [ ]:
X = tokenizer.fit_on_texts(queries)

In [ ]:
X = tokenizer.texts_to_matrix(queries)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #splitting data

In [ ]:
## Building the model

In [ ]:
def gen_model(input_size, hidden_layer_count, hidden_layer_size):
    model = Sequential()
    model.add(Dense(input_size))
    for _ in range(hidden_layer_count):
        model.add(Dense(hidden_layer_size, activation='sigmoid'))
    model.add(Dense(2))
    
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model = gen_model(X.shape[-1], 6, 128)

In [ ]:
## Training

In [ ]:
def train_model(model, epochs, batch_size, X_train, y_train, X_test, y_test):
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    y_train = np.asarray(y_train)
    y_test = np.asarray(y_test)

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
    
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=3)

    return model, test_loss, test_acc

In [ ]:
model, test_loss, test_acc = train_model(model, 5, 32, X_train, y_train, X_test, y_test)

In [ ]:
## Metrics

In [ ]:
print("Loss: ", test_loss)
print("Accuracy: ", test_acc)

In [ ]:
model.summary()

In [ ]:
## Testing

In [ ]:
def url_is_bad(url):
    X_url = tokenizer.texts_to_matrix([url])
    url_prediction = model.predict(X_url)
    return bool(np.where(url_prediction[0] == max(url_prediction[0]))[0][0])

In [ ]:
url = "/index.php?q=../../../../../../../../../etc/passwd"
print("Is bad ?", url_is_bad(url))

In [ ]:
url = "/test.php?q=data"
print("Is bad ?", url_is_bad(url))